Just exploring the package

In [1]:
import griptape
from griptape.memory.structure import ConversationMemory
from griptape.memory.tool import TextToolMemory, BlobToolMemory
from griptape.structures import Pipeline
from griptape.tasks import ToolkitTask, PromptTask
from griptape.tools import WebScraper, TextProcessor, FileManager
from griptape import utils

import os
import dotenv
import openai

dotenv.load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
text_tool_memory = TextToolMemory()
blob_tool_memory = BlobToolMemory()

In [3]:
text_processor = TextProcessor(
    memory={
        "summarize": {
            "input": [text_tool_memory]
        },
        "search": {
            "input": [text_tool_memory]
        }
    }, verbose = False
)

In [4]:
file_manager = FileManager(
    memory={
        "load_from_disk": {
            "output": [blob_tool_memory]
        },
        "save_to_disk": {
            "input": [text_tool_memory, blob_tool_memory]
        }
    }, verbose = False
)

In [5]:
web_scraper = WebScraper(
    memory={"get_content": {"output": [text_tool_memory]}}
)

In [9]:
%%time
# Pipelines represent sequences of tasks.
pipeline = Pipeline(
    memory=ConversationMemory(), 
)

pipeline.add_tasks(
    # Load up the first argument from `pipeline.run`.
    ToolkitTask(
        "{{ args[0] }}",
        tools=[web_scraper, text_processor, file_manager]
    ),
    # Augment `input` from the previous task.
    PromptTask(
        "Say the following: {{ input }}"
    )
)

# for some reason I need to reset the api_key right here, something is weird
dotenv.load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

result = pipeline.run(
    "Load https://www.griptape.ai, summarize it, and store it in griptape.txt"
)

print(
    utils.Conversation(pipeline.memory)
)

[06/12/23 14:30:33] INFO     Task a9c7896d26f7481bba0c2db372fb01e8                                                 
                             Input: Load https://www.griptape.ai, summarize it, and store it in griptape.txt

[06/12/23 14:30:37] INFO     Subtask 5aa4e03c53b94113a033ff2d5b763da3                                              
                             Thought: I can use the WebScraper tool to load the website, then use the TextProcessor
                             tool to summarize the content, and finally use the FileManager tool to save it to a   
                             file.                                                                                 
                             Action: {"type": "tool", "name": "WebScraper", "activity": "get_content", "input":    
                             {"values": {"url": "https://www.griptape.ai"}}}

                    INFO     Subtask 5aa4e03c53b94113a033ff2d5b763da3                                              
                             Observation: Output of "WebScraper.get_content" was stored in memory "TextToolMemory" 
                             with the following artifact names: b42ba7aebc4f4ade833bb650e2592f28

[06/12/23 14:30:41] INFO     Subtask ab20cfa6ed254e79b01099ca88939cc4                                              
                             Action: {"type": "tool", "name": "TextProcessor", "activity": "summarize", "input":   
                             {"values": {}, "artifacts": {"sources": [{"memory_name": "TextToolMemory",            
                             "artifact_names": ["b42ba7aebc4f4ade833bb650e2592f28"]}]} }}

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 553 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 11 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 760 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 760 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 

...

                    INFO     Subtask dafde69a6f174c179877b423a1f1d2e6                                              
                             Observation: saved successfully

[06/12/23 14:31:16] INFO     Task a9c7896d26f7481bba0c2db372fb01e8                                                 
                             Output: The website https://www.griptape.ai was summarized and saved to the file      
                             griptape.txt.

                    INFO     Task 67b724b077864601aba6512a857a74dc                                                 
                             Input: Say the following: The website https://www.griptape.ai was summarized and saved
                             to the file griptape.txt.

[06/12/23 14:31:18] INFO     Task 67b724b077864601aba6512a857a74dc                                                 
                             Output: The website https://www.griptape.ai was summarized and saved to the file      
                             griptape.txt.

Q: Load https://www.griptape.ai, summarize it, and store it in griptape.txt
A: The website https://www.griptape.ai was summarized and saved to the file griptape.txt.
CPU times: user 272 ms, sys: 20.8 ms, total: 293 ms
Wall time: 44.8 s


In [8]:
with open("griptape.txt") as f:
    print(f.read())

Data Governance and API Integrations for LLMs
Connect your internal systems to large language models safely and reliably.
Why Griptape?
Griptape offers developers the ability to build AI systems that operate across two dimensions: predictability and creativity.
For predictability, software structures like sequential pipelines or directed acyclic graphs (DAGs) are enforced. Creativity, on the other hand, is facilitated by safely prompting LLMs with tools that connect to external APIs and data. Developers can move between these two dimensions according to their use case.
Built for Developers, by Developers
Griptape is an opinionated Python framework enables developers to fully harness the potential of LLMs while enforcing strict trust boundaries, schema validation, and activity-level permissions.
Griptape can be used to create conversational and autonomous agents. Its core design tenet is to maximize the reasoning and enforce acting capabilities of LLMs, allowing developers to unleash th